In [ ]:
import nltk
import nltk.data
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import re
import spacy
import pandas as pd

from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer


nltk.download('words')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

import requests
import pandas as pd
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from textblob import TextBlob
from nltk.corpus import words
#!pip install bert-extractive-summarizer

!pip install -U transformers==3.0.0
!git clone https://github.com/patil-suraj/question_generation.git

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
Requirement already up-to-date: transformers==3.0.0 in /usr/local/lib/python3.7/dist-packages (3.0.0)
Cloning into 'question_generation'...
remote: Enumerating objects: 265, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 265 (delta 1), reused 2 (delta 0), pack-reused 259
Receiving objects: 1

In [ ]:

  
# link for extract html data
def getdata(url):
    r = requests.get(url)
    return r.text
  
def bs(link):
  htmldata = getdata(link)
  soup = BeautifulSoup(htmldata, 'html.parser')
  data = ''
  ls = []
  for data in soup.find_all("p"):
      ls.append(data.get_text())
  return ls
   

In [ ]:
cd = []
cd.append(bs("https://en.wikipedia.org/wiki/Bone_tumor"))
#thislinkisbroken _ no meaningful context in the link #cd.append(bs("https://www.medicalnewstoday.com"))
#cd.append(bs("https://www.cancer.net/cancer-types/bone-cancer-sarcoma-bone/symptoms-and-signs"))
#cd.append(bs("https://www.cancercenter.com/cancer-types/bone-cancer/symptoms"))
#cd.append(bs("https://www.healthline.com/health/bone-tumors"))
#cd.append(bs("https://www.cancer.org/cancer/bone-cancer/detection-diagnosis-staging/signs-symptoms.html"))

In [ ]:
ls = ''
for i in range (len(cd)):
  ls+=str((cd[i]))

In [ ]:
ls
mystring = ls.replace('\n', ' ').replace('[','').replace('\\','').replace("\"","").replace("'","")

for i in range(len(cd)):
  cd[i] = str(cd[i]).replace('\n', ' ').replace('[','').replace('\\','').replace("\"","").replace("'","")

  
  

In [ ]:
len(cd)

1

In [ ]:
mystring

'A bone tumor is an abnormal growth of tissue in bone, traditionally classified as noncancerous (benign) or cancerous (malignant).1]2]n, Diagnosis is generally by X-ray and other radiological tests such as CT scan, MRI, PET scan and bone scintigraphy.2] Blood tests might include a complete blood count, inflammatory markers, serum electrophoresis, PSA, kidney function and liver function.2] Urine may be sent for Bence Jones protein.2] For confirmation of diagnosis, a biopsy for histological evaluation might be required.2]n, Average five-year survival in the United States after being diagnosed with bone and joint cancer is 67%.3]n, The most common symptom of bone tumors is pain, which will gradually increase over time. A person may go weeks, months, and sometimes years before seeking help; the pain increases with the growth of the tumor. Additional symptoms may include fatigue, fever, weight loss, anemia, nausea, and unexplained bone fractures. Many patients will not experience any sympto

In [ ]:
ls = nltk.sent_tokenize(mystring)

9290

In [ ]:
ls[0]

'A bone tumor is an abnormal growth of tissue in bone, traditionally classified as noncancerous (benign) or cancerous (malignant).1]2]n, Diagnosis is generally by X-ray and other radiological tests such as CT scan, MRI, PET scan and bone scintigraphy.2] Blood tests might include a complete blood count, inflammatory markers, serum electrophoresis, PSA, kidney function and liver function.2] Urine may be sent for Bence Jones protein.2] For confirmation of diagnosis, a biopsy for histological evaluation might be required.2]n, Average five-year survival in the United States after being diagnosed with bone and joint cancer is 67%.3]n, The most common symptom of bone tumors is pain, which will gradually increase over time.'

In [ ]:
# Class initializations

nlp = spacy.load('en_core_web_sm')
stemmer = LancasterStemmer()


In [ ]:


# List to hold all input sentences
sentences = []

# Dictionary to hold sentences corresponding to respective discourse markers
disc_sentences = {}

# Remaining sentences which do not have discourse markers (To be used later to generate other kinds of questions)
nondisc_sentences = []

# List of auxiliary verbs
aux_list = ['am', 'are', 'is', 'was', 'were', 'can', 'could', 'does', 'do', 'did', 'has', 'had', 'may', 'might', 'must',
            'need', 'ought', 'shall', 'should', 'will', 'would']

# List of all discourse markers
discourse_markers = ['because', 'as a result', 'since', 'when', 'although', 'for example', 'for instance']

# Different question types possible for each discourse marker
qtype = {'because': ['Why'], 'since': ['When', 'Why'], 'when': ['When'], 'although': ['Yes/No'], 'as a result': ['Why'], 
        'for example': ['Give an example where'], 'for instance': ['Give an instance where'], 'to': ['Why']}

# The argument which forms a question
target_arg = {'because': 1, 'since': 1, 'when': 1, 'although': 1, 'as a result': 2, 'for example': 1, 'for instance': 1, 
              'to': 1}



In [ ]:
ls[:20]

['A bone tumor is an abnormal growth of tissue in bone, traditionally classified as noncancerous (benign) or cancerous (malignant).1]2]n, Diagnosis is generally by X-ray and other radiological tests such as CT scan, MRI, PET scan and bone scintigraphy.2] Blood tests might include a complete blood count, inflammatory markers, serum electrophoresis, PSA, kidney function and liver function.2] Urine may be sent for Bence Jones protein.2] For confirmation of diagnosis, a biopsy for histological evaluation might be required.2]n, Average five-year survival in the United States after being diagnosed with bone and joint cancer is 67%.3]n, The most common symptom of bone tumors is pain, which will gradually increase over time.',
 'A person may go weeks, months, and sometimes years before seeking help; the pain increases with the growth of the tumor.',
 'Additional symptoms may include fatigue, fever, weight loss, anemia, nausea, and unexplained bone fractures.',
 'Many patients will not experien

In [ ]:
cv=[]
for i in ls : 
    if (len(i)>40):
        cv.append(i)
cs = ''
for i in ls : 
    if (len(i)>40):
        cs+=i
# this is cv , a list to store paragraphs and cs to generate wordcloud

In [ ]:
# this section is cleaning text to view 
my_stopwords = nltk.corpus.stopwords.words('english')
word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'


def remove_links(tweet):
    '''Takes a string and removes web links from it'''
    tweet = re.sub(r'http\S+', '', tweet) # remove http links
    
    tweet = re.sub(r'bit.ly/\S+', '', tweet) # rempve bitly links
    tweet = tweet.strip('[link]') # remove [links]
    return tweet

def remove_users(tweet):
    '''Takes a string and removes retweet and @user information'''
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove retweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove tweeted at
    return tweet
def cleanTxt(tweet, bigrams=False):


    tweet = tweet.lower() # lower case
    tweet = re.sub('['+my_punctuation + ']+', ' ', tweet) # strip punctuation
    tweet = re.sub('\s+', ' ', tweet) #remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet) # remove numbers
    tweet_token_list = [word for word in tweet.split(' ')
                            if word not in my_stopwords] # remove stopwords

    tweet_token_list = [word_rooter(word) if '#' not in word else word
                        for word in tweet_token_list] # apply word rooter
    final = []

    for x in tweet_token_list:
      if x in words.words():
        final.append(x)
    tweet_token_list = final

    if bigrams:
        tweet_token_list = tweet_token_list+[tweet_token_list[i]+'_'+tweet_token_list[i+1]
                                            for i in range(len(tweet_token_list)-1)]
    tweet = ' '.join(tweet_token_list)
    return tweet


In [ ]:
def cleant(tweet):


    tweet = tweet.lower() # lower case
    tweet = re.sub('['+my_punctuation + ']+', ' ', tweet) # strip punctuation
    tweet = re.sub('\s+', ' ', tweet) #remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet) # remove numbers
    return tweet




In [ ]:
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [ ]:
'''
!pip install rake-nltk

from rake_nltk import Rake


def replaceIC(word, sentence):
  insensitive_hippo = re.compile(re.escape(word), re.IGNORECASE)
  return insensitive_hippo.sub('__________________', sentence)
'''

"\n!pip install rake-nltk\n\nfrom rake_nltk import Rake\n\n\ndef replaceIC(word, sentence):\n  insensitive_hippo = re.compile(re.escape(word), re.IGNORECASE)\n  return insensitive_hippo.sub('__________________', sentence)\n"

In [ ]:
for i in range(len(cv)):
  cv[i] = str(cv[i])

In [ ]:
bs = [] # make a new list
for i in range(len(cv)):
  if (len(cv[i])>170):
    bs.append(cv[i])

cv = bs

In [ ]:
'''
for i in cv:
   lss = nltk.tokenize.sent_tokenize(i)
   for j in lss:
     j = cleant(j)
     r = Rake() 
     r.extract_keywords_from_text(j)

     X = r.get_ranked_phrases()
     
     if(len(X)>1):
       print("Question()-" , end=" ")
       print(replaceIC(X[0],j))
       print("Ans()-",end=" ")
       print(X[0])
       text = j
       blob = TextBlob(text)
       blob.tags           # [('The', 'DT'), ('titular', 'JJ'),
                            #  ('threat', 'NN'), ('of', 'IN'), ...]

       blob.noun_phrases   # WordList(['titular threat', 'blob',
                            #            'ultimate movie monster',
                            #            'amoeba-like mass', ...])

       for sentence in blob.sentences:
         print(sentence.sentiment)
          
       print("----------------------------------------------------------------------------------------------------------------------")
  
'''


'\nfor i in cv:\n   lss = nltk.tokenize.sent_tokenize(i)\n   for j in lss:\n     j = cleant(j)\n     r = Rake() \n     r.extract_keywords_from_text(j)\n\n     X = r.get_ranked_phrases()\n     \n     if(len(X)>1):\n       print("Question()-" , end=" ")\n       print(replaceIC(X[0],j))\n       print("Ans()-",end=" ")\n       print(X[0])\n       text = j\n       blob = TextBlob(text)\n       blob.tags           # [(\'The\', \'DT\'), (\'titular\', \'JJ\'),\n                            #  (\'threat\', \'NN\'), (\'of\', \'IN\'), ...]\n\n       blob.noun_phrases   # WordList([\'titular threat\', \'blob\',\n                            #            \'ultimate movie monster\',\n                            #            \'amoeba-like mass\', ...])\n\n       for sentence in blob.sentences:\n         print(sentence.sentiment)\n          \n       print("----------------------------------------------------------------------------------------------------------------------")\n  \n'

In [ ]:

%cd question_generation
from pipelines import pipeline




/content/question_generation/question_generation


In [ ]:
nlp = pipeline("question-generation")

In [ ]:
# this is a model used to summarize big paragraph 
'''

from summarizer import Summarizer

model = Summarizer()
for i in range(len(cd)):
  cd[i] = model(str(cd[i]))

'''

'\n\nfrom summarizer import Summarizer\n\nmodel = Summarizer()\nfor i in range(len(cd)):\n  cd[i] = model(str(cd[i]))\n\n'

#Creating Q and A , to csv part 2 . 

In [ ]:
cd[0]

'A bone tumor is an abnormal growth of tissue in bone, traditionally classified as noncancerous (benign) or cancerous (malignant).1]2]n, Diagnosis is generally by X-ray and other radiological tests such as CT scan, MRI, PET scan and bone scintigraphy.2] Blood tests might include a complete blood count, inflammatory markers, serum electrophoresis, PSA, kidney function and liver function.2] Urine may be sent for Bence Jones protein.2] For confirmation of diagnosis, a biopsy for histological evaluation might be required.2]n, Average five-year survival in the United States after being diagnosed with bone and joint cancer is 67%.3]n, The most common symptom of bone tumors is pain, which will gradually increase over time. A person may go weeks, months, and sometimes years before seeking help; the pain increases with the growth of the tumor. Additional symptoms may include fatigue, fever, weight loss, anemia, nausea, and unexplained bone fractures. Many patients will not experience any sympto

In [ ]:
# data instance is our dictionary to store all the Questions 
data = {
  "Question": [],
  "Ans": [],
  "context":[]
}

#load data into a DataFrame object:
df = pd.DataFrame(data)

print(df) 

Empty DataFrame
Columns: [Question, Ans, context]
Index: []


In [ ]:
#df.Question = output["questions"][0]["Question"]
#df.Ans = output["questions"][0]["Answer"]

In [ ]:
# nltk tokenizing sentence from paragraph 
# this gives us simple sentences , from all the paragraph 
cv =[]
bc =[]
for i in cd :
  bc = nltk.sent_tokenize(i)
  
  cv.extend(bc)

In [ ]:
len(cv[0])

724

In [ ]:

# This is a filter step , for paragraph

lss = ''
bc = []
len(cv[1])
for i in cv:
  if (len(i))<200 :  # iF THE LENGTH OF paragraph is smaller then 200 ,  then we increase size of list as following 
    if((len(lss)) < 200 ): # lss is a string , to help us manipulate strings , if lss len is less then 200 , 
      lss += str(i) # we append the list i which is smaller to lss , and we will append multiple strings from tokenize sentence to increase the length 
    else:
      bc.append(lss) # if lss is not small anymore , then else loop runs and it appends the string to our final list which we pass to model , bc 
      lss =str(i)  # now we change lss , to the next sentence modelling after saving the previos lss string to our final list of Q
  else:
    bc.append(str(i)) # this runs when the string is big enough that it doesn't need to be added or increased in size

      



258

In [ ]:
nlp = pipeline("e2e-qg")

In [ ]:
d = []
for i in range (5):#(len(bc)):
  d.extend(nlp(cv[i]))

In [ ]:
xx = nlp(cv[2])

In [ ]:
type(xx)

list

# Main

In [ ]:

d= []
for i in range (len(cv)):
  d.clear()
  try:
    d = nlp(cv[i]) # runs the loop model to genereate answer and Question while passing the final list to our model , bc is final list
  except:
    continue



 
  for j in range(len(d)): # append the list of Q to our data , dictionry to generate csv later 

    try:
      data["Question"].append(d[j])
      data["Ans"].append("")
      data["context"].append(cv[i])
    except:
      continue
 
  



In [ ]:
df2 = pd.DataFrame(data)
df2.drop(labels="Ans",axis=1,inplace=True)


# New Section

In [ ]:
df2.to_csv('QandA.csv', index=False)


In [ ]:
import pandas as pd
df= pd.read_csv("QandA.csv")


In [ ]:
for i in range(10):
  print(df.Question[i])

{'answer': 'pain', 'question': 'What is the most common symptom of bone tumors?'}
{'answer': 'weeks, months, and sometimes years', 'question': 'How long can a person go before seeking help?'}
{'answer': 'fatigue, fever, weight loss, anemia, nausea, and unexplained bone fractures', 'question': 'What other symptoms may be associated with fatigue, fever, weight loss, anemia, nausea, and unexplained bone fractures?'}
{'answer': 'painless', 'question': 'What type of mass will many patients experience?'}
{'answer': 'pathologic fractures', 'question': 'What causes bone tumors to weaken the structure of the bone?'}
{'answer': 'secondary', 'question': 'What type of tumors can be classified as?'}
{'answer': '50 to 100 times', 'question': 'How many times more common are secondary malignant bone tumors?'}
{'answer': 'neoplastic, developmental, traumatic, infectious, or inflammatory', 'question': 'Common benign bone tumors may be what in etiology?'}
{'answer': 'Common benign bone tumors', 'question

In [ ]:

from google.colab import files
files.download("QandA.csv")

In [ ]:
cont = []
cont.append(str(cd[0]))
nlp({
    "question": "What is 42 ?",
    "context": "42 is the answer to life, the universe and everything."
})

AttributeError: ignored

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
